In [1]:
%cd ..

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from ipywidgets import interact

import cufflinks as cf
cf.go_offline(connected=True)

import bokeh.io
bokeh.io.output_notebook()

np.random.seed(42)

/home/rosneft_user_2500/anomaly-detection


Loading BokehJS ...

In [2]:
root_folder = %pwd
import sys
sys.path = [root_folder] + sys.path

# Загрузка данных

In [3]:
from sklearn.model_selection import train_test_split
from src.features.build_features import rolling_window

prediction_len = 1
window_len = 32
batch_size = 32

data = pd.read_csv('data/processed/tep_data.csv', index_col='Index')
print(f'Len of dataset: {data.shape[0]}')

Len of dataset: 12801


## Decomposition

In [4]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import acf, pacf

@interact(component=(0, 40))
def myacf(component):
    plot_acf(data.values[:, component], lags=np.arange(0, 2000))

interactive(children=(IntSlider(value=20, description='component', max=40), Output()), _dom_classes=('widget-i…

In [5]:
import statsmodels.tsa.seasonal as seasonal
period = 750
decomposed = seasonal.seasonal_decompose(data.values,
                                         period=period,
                                         extrapolate_trend='freq')


@interact(comp=(0, 40))
def f(comp):
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.title('Trend')
    plt.plot(decomposed.trend[:, comp])
    plt.subplot(1, 2, 2)
    plt.title('Seasonal')
    plt.plot(decomposed.seasonal[:, comp])

interactive(children=(IntSlider(value=20, description='comp', max=40), Output()), _dom_classes=('widget-intera…

# Обучение

In [6]:
from datetime import datetime
def get_log_path(name):
    return name + '_' + datetime.now().strftime('%Y-%m-%d-%H-%M')

In [7]:
import matplotlib.pyplot as plt

def compare_plot_1d(real, real_lbl, pred, pred_lbl, shift_pred=0):
    plt.plot(real, label=real_lbl, alpha=0.7)
    plt.plot(list(range(shift_pred, real.shape[0])), pred, label=pred_lbl, alpha=0.8)
    plt.legend()

def compare_plot_from_2d(real, real_lbl, pred, pred_lbl, component, shift_pred=0):
    real_comp = real[:, component]
    pred_comp = pred[:, component]
    compare_plot_1d(real_comp,real_lbl, pred_comp, pred_lbl, shift_pred)
    
def compare_plot_from_2d_wrapper(*args, **kwargs):
    def f(**inner_kwargs):
        return compare_plot_from_2d(*args, **kwargs, **inner_kwargs)
    return f

def interactive_plotter_by_component(funcs, titles, suptitle):
    @interact(component=(0, data.shape[1]-1))
    def __inner__(component):
        plt.figure(figsize=(18, 8))
        plt.suptitle(suptitle)
        for i, (func, title) in enumerate(zip(funcs, titles), 1):
            plt.subplot(1, len(funcs), i)
            plt.title(title)
            func(component=component)

## Тренд

### Подготовка данных

In [8]:
from src.models.torch.utils import to_dataloader

X_trend = rolling_window(decomposed.trend, window_len)[:-prediction_len]
y_trend = rolling_window(decomposed.trend, prediction_len, window_len)

X_tr, X_te, y_tr, y_te = train_test_split(X_trend, y_trend, train_size=0.7, shuffle=False)

train_set = to_dataloader(X_tr, y_tr, dict(batch_size=batch_size))
test_set = to_dataloader(X_te, y_te, dict(batch_size=batch_size))

### Обучение

In [8]:
import torch
from src.models.torch.models import LSTM, Trainer

config = dict(
    input_size=X_tr[0].shape[1],
    hidden_size=16,
    num_layers=1,
    batch_first=True,
    bidirectional=True,
)

device = torch.device('cpu')
model = LSTM(**config).to(device)
criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim,
                                                       patience=3,
                                                       threshold=0.01)

trainer = Trainer(
    model,
    criterion,
    optim,
    scheduler,
    device,
    get_log_path(
        f'logs/trend-{config["num_layers"]}-layers-{config["hidden_size"]}-hidden-{window_len}-len'
    ),
    stateful=True)

In [10]:
# # for test purpouses
# sz = 200
# xx = torch.rand(sz, window_len, data.shape[1])
# yy = torch.rand(sz, data.shape[1])
# xxdatayy = to_dataloader(xx, yy, dict(batch_size=batch_size))
# trainer.train(xxdatayy, xxdatayy, 5)

In [10]:
trainer.train(train_set, test_set, 15)

Epoch 0 of train: :   0%|          | 0/280 [00:00<?, ?it/s, loss=0.66]/home/rosneft_user_2500/anomaly-detection/src/models/torch/utils.py:25: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

Epoch 0 of train: :   1%|          | 2/280 [00:00<00:22, 12.52it/s, loss=0.488]

Epoch 0/14
----------


Epoch 1 of train: :   1%|          | 2/280 [00:00<00:17, 15.90it/s, loss=0.185]

Loss: 0.0601

Epoch 1/14
----------


Epoch 2 of train: :   1%|          | 2/280 [00:00<00:14, 19.27it/s, loss=0.0812]

...

Loss: 0.0356

Epoch 13/14
----------


Epoch 14 of train: :   1%|          | 3/280 [00:00<00:13, 21.14it/s, loss=0.0202]

Loss: 0.0254

Epoch 14/14
----------


Epoch 14 of val: : 100%|██████████| 120/120 [00:06<00:00, 18.92it/s, loss=0.149] 


Loss: 0.0247



In [11]:
torch.save(model, 'trend.pth')

### Предсказание

In [9]:
import torch
model = torch.load('trend.pth')

/home/rosneft_user_2500/anaconda3/envs/anom-det/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning:

source code of class 'src.models.torch.models.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.



In [10]:
def forecast_tr_te(model, X_tr, X_te, batch_size):
    model.reset_states()
    if len(X_tr) % batch_size:
        raise ValueError('Len of X_tr must be divisible by batch size')
    train_pred = model.forecast(X_tr, batch_size)
    test_pred = model.forecast(X_te, batch_size)
    return train_pred, test_pred

In [11]:
train_pred, test_pred = forecast_tr_te(model, X_tr, X_te, 41)

train_pred = train_pred.detach().numpy()
test_pred = test_pred.detach().numpy()

  0%|          | 1/218 [00:00<00:55,  3.89it/s]/home/rosneft_user_2500/anomaly-detection/src/models/torch/models.py:45: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

100%|██████████| 94/94 [00:02<00:00, 43.05it/s]


In [12]:
split_point = len(X_tr) + window_len

interactive_plotter_by_component([
    compare_plot_from_2d_wrapper(decomposed.trend[:split_point],
                                 'real',
                                 train_pred,
                                 'pred',
                                 shift_pred=window_len),
    compare_plot_from_2d_wrapper(decomposed.trend[split_point:], 'real',
                                 test_pred, 'pred')
], ['Train', 'Test'], 'Next value prediction of trend')

interactive(children=(IntSlider(value=20, description='component', max=40), Output()), _dom_classes=('widget-i…

#### Weighted MSE - Optional

На некоторых компонентах плохое предсказание, поэтому им нужно получить больше веса

In [18]:
# from sklearn.metrics import mean_squared_error

# width = 3000
# err = mean_squared_error(test_pred[:width], decomposed.trend[split_point:split_point+width], multioutput='raw_values')
# plt.bar(np.arange(len(err)), err)

# def weighted_mse_loss(weights):
#     weights = torch.tensor(weights)
#     criterion = torch.nn.MSELoss(reduction='none')
#     def mse(input, target):
#         nonlocal weights, criterion
#         loss = criterion(input, target)
#         loss = loss * weights.expand_as(loss)
#         return loss.mean()
#     return mse

# criterion = weighted_mse_loss(err / err.sum())
# optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim,
#                                                        patience=3,
#                                                        threshold=0.01)

# trainer = Trainer(
#     model,
#     criterion,
#     optim,
#     scheduler,
#     device,
#     get_log_path(
#         f'logs/trend-retraining-{config["num_layers"]}-layers-{config["hidden_size"]}-hidden-{window_len}-len'
#     ),
#     stateful=True)



# model.reset_states()
# trainer.train(train_set, test_set, 10)

# model.reset_states()
# split_point = len(X_tr) + window_len
# train_pred = next_value_prediction(model, decomposed.trend[:split_point],
#                                    window_len)
# test_pred = next_value_prediction(model, decomposed.trend[split_point:],
#                                   window_len)

# train_pred = train_pred.detach().numpy()
# test_pred = test_pred.detach().numpy()

## Предсказание остатков

### Подоготвка данных 

In [17]:
train_resid = np.array(y_tr).squeeze() - train_pred
test_resid = np.array(y_te).squeeze() - test_pred

trend_pred = np.r_[train_pred, test_pred]
residuals = np.r_[train_resid, test_resid]

In [18]:
interactive_plotter_by_component([
    compare_plot_from_2d_wrapper(decomposed.trend,
                                 'real',
                                 trend_pred,
                                 'Next value prediction (LSTM)',
                                 shift_pred=window_len),
    lambda component: plt.plot(residuals[:, component], label='Residuals (data - trend_pred)')
],
['Trend', 'Resid'], None)

interactive(children=(IntSlider(value=20, description='component', max=40), Output()), _dom_classes=('widget-i…

In [19]:
from scipy.ndimage import gaussian_filter1d
@interact(component=(0, 40), sigma=np.arange(1, 11))
def plot(component, sigma):
    smoothed = gaussian_filter1d(train_resid, sigma, axis=0)
    pd.DataFrame(dict(component=smoothed[:, component])).iplot()

interactive(children=(IntSlider(value=20, description='component', max=40), Dropdown(description='sigma', opti…

In [20]:
sigma = 10
smoothed_train = gaussian_filter1d(train_resid, sigma, axis=0)
smoothed_test = gaussian_filter1d(test_resid, sigma, axis=0)

In [34]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler().fit(train_resid)
# train_resid_scaled = ss.transform(smoothed_train)
# test_resid_scaled = ss.transform(smoothed_test)

### Обучение

In [62]:
train_set = to_dataloader(X_tr, smoothed_train, dict(batch_size=batch_size))
test_set = to_dataloader(X_te, smoothed_test, dict(batch_size=batch_size))

In [63]:
import torch
from src.models.torch.models import LSTM, Trainer

config = dict(
    input_size=X_tr[0].shape[1],
    hidden_size=16,
    num_layers=1,
    batch_first=True,
    bidirectional=True,
)

device = torch.device('cpu')
model = LSTM(**config).to(device)
criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim,
                                                       patience=3,
                                                       threshold=0.01)

trainer = Trainer(
    model,
    criterion,
    optim,
    scheduler,
    device,
    get_log_path(
        f'logs/resid-smoothed-{config["num_layers"]}-layers-{config["hidden_size"]}-hidden-{window_len}-len'
    ),
    stateful=True)

In [64]:
trainer.train(train_set, test_set, 10)

Epoch 0 of train: :   0%|          | 0/280 [00:00<?, ?it/s, loss=0.104]/home/rosneft_user_2500/anomaly-detection/src/models/torch/utils.py:25: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

Epoch 0 of train: :   1%|          | 2/280 [00:00<00:25, 11.08it/s, loss=0.0621]

Epoch 0/9
----------


Epoch 1 of train: :   0%|          | 1/280 [00:00<00:28,  9.65it/s, loss=0.0389]

Loss: 0.0240

Epoch 1/9
----------


Epoch 2 of train: :   1%|          | 2/280 [00:00<00:24, 11.27it/s, loss=0.0314]

...

Loss: 0.0187

Epoch 8/9
----------


Epoch 9 of train: :   1%|          | 3/280 [00:00<00:14, 19.78it/s, loss=0.00204]

Loss: 0.0182

Epoch 9/9
----------


Epoch 9 of val: : 100%|██████████| 120/120 [00:07<00:00, 16.90it/s, loss=0.109]  


Loss: 0.0181



In [65]:
torch.save(model, 'resid.pth')

### Prediction

In [14]:
resid_model = torch.load('resid.pth')
resid_model.reset_states()

train_resid_pred, test_resid_pred = forecast_tr_te(
    resid_model, X_tr, X_te, 41)

  0%|          | 0/218 [00:00<?, ?it/s]/home/rosneft_user_2500/anomaly-detection/src/models/torch/models.py:45: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

100%|██████████| 94/94 [00:02<00:00, 44.07it/s]


In [15]:
train_resid_pred = train_resid_pred.detach().numpy()
test_resid_pred = test_resid_pred.detach().numpy()

In [21]:
interactive_plotter_by_component([
    compare_plot_from_2d_wrapper(np.r_[train_resid_pred, test_resid_pred], 'Next value prediction (LSTM)',
                                 np.r_[smoothed_train, smoothed_test], 'Residuals (data - trend_pred)')
], ['Residuals'], None)

interactive(children=(IntSlider(value=20, description='component', max=40), Output()), _dom_classes=('widget-i…